<a href="https://colab.research.google.com/github/Paras-Shirvale/Quiz-2/blob/main/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [4]:
!pip install seaborn

In [5]:
!pip install wandb

In [7]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from PIL import Image
import os

In [9]:
# Specify the folder where images are stored; update this path as needed
# cats = "test\cats"
# dogs = "test\dogs"

cats = "/content/drive/MyDrive/test/cats"
dogs = "/content/drive/MyDrive/test/dogs"

# List image files with .jpg extension
cats_files = [f for f in os.listdir(cats) if f.lower().endswith('.jpg')]
dogs_files = [f for f in os.listdir(dogs) if f.lower().endswith('.jpg')]

# Create a dataframe from the list of image filenames
df_cats = pd.DataFrame(cats_files, columns=['cats_filename'])
df_cats['cats_filepath'] = df_cats['cats_filename'].apply(lambda x: os.path.join(cats, x))
df_cats['cats_filename'] = df_cats['cats_filename'].str.slice(0, -4)
df_cats['cats_pixels'] = df_cats['cats_filepath'].apply(lambda x: Image.open(x))

df_dogs = pd.DataFrame(dogs_files, columns=['dogs_filename'])
df_dogs['dogs_filepath'] = df_dogs['dogs_filename'].apply(lambda x: os.path.join(dogs, x))
df_dogs['dogs_filename'] = df_dogs['dogs_filename'].str.slice(0, -4)
df_dogs['dogs_pixels'] = df_dogs['dogs_filepath'].apply(lambda x: Image.open(x))

# Sorted the Dataframe by filename
df_cats = df_cats.sort_values(by='cats_filename', ascending=True)
df_dogs = df_dogs.sort_values(by='dogs_filename', ascending=True)

In [10]:
# Split df_cats and df_dogs into training and test sets (80% train, 20% test)
df_cats_train, df_cats_test = train_test_split(df_cats, test_size=0.2, random_state=42)
df_dogs_train, df_dogs_test = train_test_split(df_dogs, test_size=0.2, random_state=42)

print("df_cats_train shape:", df_cats_train.shape)
print("df_cats_test shape:", df_cats_test.shape)
print("df_dogs_train shape:", df_dogs_train.shape)
print("df_dogs_test shape:", df_dogs_test.shape)

df_cats_train shape: (56, 3)
df_cats_test shape: (14, 3)
df_dogs_train shape: (56, 3)
df_dogs_test shape: (14, 3)


In [11]:
# Specify the save directory
save_dir = "/"

# Use the save_dir variable correctly in the file path
df_cats_train.to_csv(f'{save_dir}/cats_train.csv', index=False)
df_dogs_train.to_csv(f'{save_dir}/dogs_train.csv', index=False)
df_cats_test.to_csv(f'{save_dir}/cats_test.csv', index=False)
df_dogs_test.to_csv(f'{save_dir}/dogs_test.csv', index=False)

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import seaborn as sns
import wandb
from PIL import Image
import os

# Initialize Weights & Biases
wandb.init(project="image-classification-report")

# Load the data (assuming you have preprocessed data as CSV files)
df_cats_train = pd.read_csv('/cats_train.csv')
df_dogs_train = pd.read_csv('/dogs_train.csv')
df_cats_test = pd.read_csv('/cats_test.csv')
df_dogs_test = pd.read_csv('/dogs_test.csv')


# Combine and preprocess data (example - adapt to your actual data)
# Assuming 'cats_pixels' column contains pixel data (replace with your actual feature)
def extract_pixel_data(filepath):
    try:
        img = Image.open(filepath).resize((64,64))
        return np.array(img).flatten()
    except Exception as e:
        print(f"Error processing image: {filepath}, error: {e}")
        return np.zeros(64*64*3)


df_cats_train['pixel_data'] = df_cats_train['cats_filepath'].apply(extract_pixel_data)
df_dogs_train['pixel_data'] = df_dogs_train['dogs_filepath'].apply(extract_pixel_data)
df_cats_test['pixel_data'] = df_cats_test['cats_filepath'].apply(extract_pixel_data)
df_dogs_test['pixel_data'] = df_dogs_test['dogs_filepath'].apply(extract_pixel_data)

X_train = np.concatenate([df_cats_train['pixel_data'].values, df_dogs_train['pixel_data'].values])
y_train = np.concatenate([np.zeros(len(df_cats_train)), np.ones(len(df_dogs_train))])

X_test = np.concatenate([df_cats_test['pixel_data'].values, df_dogs_test['pixel_data'].values])
y_test = np.concatenate([np.zeros(len(df_cats_test)), np.ones(len(df_dogs_test))])


# Model definition
model = models.Sequential([
    layers.Input(shape=(12288,)),  # Input dimension
    layers.Dense(16, activation='relu'), # Hidden layer
    layers.Dense(2, activation='softmax')  # Output layer
])

# Hyperparameters
learning_rate = 0.001  # Example - replace with the best learning rate from Task 2
batch_size = 32       # Example - replace with the best batch size from Task 2

# Log hyperparameters to wandb
wandb.config.learning_rate = learning_rate
wandb.config.batch_size = batch_size
wandb.config.architecture = "2-layer NN (1 hidden layer)"

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model and log metrics
history = model.fit(X_train, y_train, epochs=3, batch_size=batch_size, validation_split=0.2)

# Log training and validation loss per epoch to wandb
for epoch in range(3):
    wandb.log({"train_loss": history.history['loss'][epoch],
               "val_loss": history.history['val_loss'][epoch],
               "epoch": epoch})

# Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=1)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

#Log metrics to wandb
wandb.log({"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1})

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Log confusion matrix to wandb
wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None, y_true=y_test, preds=y_pred)})


# Plot and log loss curves to wandb
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
wandb.log({"loss_curves": plt})


# Finish the wandb run
wandb.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: paras-shirvale (paras-shirvale-indian-institute-of-technology-gandhinagar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


ValueError: Invalid dtype: object